In [2]:
#Bibliotecas para manipulação dos dados e de tempo
import pandas as pd
import numpy as np
import datetime as dt

import numpy as np
from pandas import read_excel
import matplotlib.mlab as mlab
import matplotlib.pyplot as plt
import seaborn as sns
import statistics as st
import scipy.stats as stats
import statsmodels.api as sm
import re

import time
from dateutil import relativedelta as r


'''
O problema aqui é pegar cada número de telefone e avaliar as datas mínimas e máximas por linhas removendo 
datas que possuem intervalo.

'''

In [7]:
#Fazendo a importação da base de dados
csu = pd.read_csv('C:/Users/eden.santana/Desktop/Datasets/CSU/202002.Unidas.Ativo_1.csv', sep=';', 
                 encoding='utf-8', low_memory = False)

In [9]:
#Leitura das três primeiras linhas
csu.head(3)

,ChaveUnica,CPF_Cliente,Cod,Tabulacao,Matricula,Operador,Data,Telefone,Campanha,Classificacao,Mailing,Unic
0,1647639,13318538850,2,NoAnswer,NaN,NaN,2020-02-01 12:15:26.407,12988081488,UNIDAS - ASSINATURA DE VEICULOS,5,Mailing_Unidas_Assinatura_Veiculos_21012020_Co...,1
1,1721342,30956474845,8,CarrierMessage,NaN,NaN,2020-02-20 15:29:19.080,14991201178,UNIDAS – LEAD MACHINE,5,Mailing_LeadMachine_20022020,1
2,1721342,30956474845,8,CarrierMessage,NaN,NaN,2020-02-27 14:52:10.970,14991201178,UNIDAS – LEAD MACHINE,5,Mailing_LeadMachine_20022020,2


In [11]:
#Convertendo a coluna de Data para timestamp
csu['timestamp'] = pd.to_datetime(csu['Data'], infer_datetime_format = True)

In [12]:
#Leitura das três primeiras linhas
csu.head(3)

,ChaveUnica,CPF_Cliente,Cod,Tabulacao,Matricula,Operador,Data,Telefone,Campanha,Classificacao,Mailing,Unic,timestamp
0,1647639,13318538850,2,NoAnswer,NaN,NaN,2020-02-01 12:15:26.407,12988081488,UNIDAS - ASSINATURA DE VEICULOS,5,Mailing_Unidas_Assinatura_Veiculos_21012020_Co...,1,2020-02-01 12:15:26.407
1,1721342,30956474845,8,CarrierMessage,NaN,NaN,2020-02-20 15:29:19.080,14991201178,UNIDAS – LEAD MACHINE,5,Mailing_LeadMachine_20022020,1,2020-02-20 15:29:19.080
2,1721342,30956474845,8,CarrierMessage,NaN,NaN,2020-02-27 14:52:10.970,14991201178,UNIDAS – LEAD MACHINE,5,Mailing_LeadMachine_20022020,2,2020-02-27 14:52:10.970


In [19]:
#Verificanco o tipo da coluna, podemos observar que a nova coluna timestamp está como data no formato pandas
csu.dtypes

ChaveUnica                int64
CPF_Cliente              object
Cod                       int64
Tabulacao                object
Matricula               float64
Operador                 object
Data                     object
Telefone                  int64
Campanha                 object
Classificacao             int64
Mailing                  object
Unic                      int64
timestamp        datetime64[ns]
dtype: object

In [41]:
#Fazendo o agrupamento pelos números de telefone calculando as datas mínimas e datas máximas
x = csu.groupby(['Telefone']).agg({'timestamp' : [np.min, np.max]})
csu.groupby(['Telefone']).agg({'timestamp' : [np.min, np.max]})

timestamp                        
                               amin                    amax
Telefone                                                   
1120155702  2020-02-05 17:41:21.967 2020-02-05 17:41:21.967
1120183214  2020-02-04 14:48:16.270 2020-02-04 14:48:16.270
1120325432  2020-02-11 17:35:17.740 2020-02-13 09:09:46.097
1120519371  2020-02-04 14:49:19.247 2020-02-04 14:49:19.247
1120561421  2020-02-11 17:42:02.507 2020-02-13 09:18:42.067
...                             ...                     ...
99999346315 2020-02-01 10:43:17.583 2020-02-03 15:00:55.500
99999770448 2020-02-13 14:40:27.153 2020-02-15 14:36:35.283
99999776373 2020-02-11 10:21:57.587 2020-02-12 17:36:53.407
99999779030 2020-02-13 18:38:34.370 2020-02-13 18:38:34.370
99999999999 2020-02-05 09:38:59.183 2020-02-17 11:38:15.290

[49426 rows x 2 columns]

In [42]:
#Calculando as diferenças entre as datas mínimas e datas máximas
x[('timestamp', 'n_dias')] = (x[('timestamp', 'amax')] - x[('timestamp', 'amin')]) / pd.Timedelta(days=1)

'''Timedeltas são diferenças nos tempos, expressas em unidades de 
diferença, para dias, horas, minutos, segundos. Eles podem ser positivos e negativos.'''

In [43]:
#Visualização dos cinco últimos valores
x.tail()

timestamp                                   
                               amin                    amax     n_dias
Telefone                                                              
99999346315 2020-02-01 10:43:17.583 2020-02-03 15:00:55.500   2.178911
99999770448 2020-02-13 14:40:27.153 2020-02-15 14:36:35.283   1.997316
99999776373 2020-02-11 10:21:57.587 2020-02-12 17:36:53.407   1.302035
99999779030 2020-02-13 18:38:34.370 2020-02-13 18:38:34.370   0.000000
99999999999 2020-02-05 09:38:59.183 2020-02-17 11:38:15.290  12.082825

In [46]:
'''Exportando os valores para o formato excel, aqui o índice foi ativado porque os 
      números de telefone estão como índice '''

x.to_excel('csu.xlsx', index=True)